# Housing Days On Market PCA Feature and Model Selection

## Information

Housing related data sources were combined in the project SQLite database. The output CSV file is analyzed here. 

### Environment Information:

Environment used for coding is as follow:

Oracle VM VirtualBox running Ubuntu (guest) on Windows 10 (host).

Current conda install:

               platform : linux-64
          conda version : 4.2.13
       conda is private : False
      conda-env version : 4.2.13
    conda-build version : 1.20.0
         python version : 2.7.11.final.0
       requests version : 2.9.1
       
Package requirements:

dill : 0.2.4, numpy : 1.11.2, pandas : 0.19.1, matplotlib : 1.5.1, scipy : 0.18.1, seaborn : 0.7.0, scikit-image : 0.12.3, scikit-learn : 0.18.1


## Python Package(s) Used

In [1]:
import dill
import numpy as np
import pandas as pd
import time

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, SGDRegressor, HuberRegressor, PassiveAggressiveRegressor, TheilSenRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score   
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [4]:
%matplotlib inline

In [5]:
plt.style.use('seaborn-whitegrid')

## Data and Methods

### Data Fetching

In [6]:
# Import data csv into dataframe
df = pd.read_csv('df_prep_for_feature_selection_output.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.head()

,ListPrice2,Bedrooms,BathsFull,BathsHalf,Levels,Fireplaces,BasementY/N,Acres,YearBuilt,DOMP,...,HSSR_1.0,HSSR_2.0,HSSR_3.0,HSSR_4.0,MSSR_0.0,MSSR_1.0,MSSR_2.0,MSSR_3.0,MSSR_4.0,MSSR_5.0
0,269900,3,2,0.0,3,0,1,0.082,1950,10,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,255000,2,2,1.0,2,1,0,0.000,1987,41,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,299900,1,1,0.0,1,0,0,0.000,1941,12,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,245000,1,1,0.0,1,0,0,0.000,1941,16,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,250000,1,1,0.0,1,0,0,0.000,1891,5,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Regression Modelling

### Feature Selection

In [7]:
df_2 = df.copy()

In [8]:
# Performing feature selection on full dataset, resulted in best_score ~ 0.1.
# Qcutting to separate out data.
# qcut value = 1 is dataset as is.

# Qcut DOMP target data
df_2['qcut_DOMP'] = pd.qcut(df_2['DOMP'], 1, labels = False)
# Print out total row counts for each group
print(df_2['qcut_DOMP'].value_counts())
# Select specific range
df_2 = df_2[df_2['qcut_DOMP'] == 0]
# Save dataframe to disk
df_2.to_csv('df_feature_selection_save_point.csv')

0    13725
Name: qcut_DOMP, dtype: int64


In [9]:
# Copy dataframe for dropping columns and determining categorical columns
df_3 = df_2.copy()
# Drop target column and qcut column for test-train-split
df_3 = df_3.drop('DOMP', axis=1)
df_3 = df_3.drop('qcut_DOMP', axis=1)

In [ ]:
# Checking for grouped categorical columns
#df_3.columns[93:178]

In [ ]:
# Columns that are not scaled since they are categorical
# cat_df = df_3[[u'BasementY/N']]
# cat_df_2 = df_3.ix[:,39:43]
# cat_df_3 = df_3.ix[:,48:52]
# cat_df_4 = df_3.ix[:,57:61]
# cat_df_5 = df_3.ix[:,93:178]
# cat_df_6 = pd.concat([cat_df,cat_df_2,cat_df_3,cat_df_4,cat_df_5],axis=1)
# CATEGORICAL = [x for x in cat_df_6.columns]

In [10]:
CATEGORICAL = ['BasementY/N','ES_IsCharter','ES_IsMagnet','ES_IsTitleI','ES_IsVirtual',
               'HS_IsCharter','HS_IsMagnet','HS_IsTitleI','HS_IsVirtual','MS_IsCharter',
               'MS_IsMagnet','MS_IsTitleI','MS_IsVirtual','zip_20001','zip_20002','zip_20004',
               'zip_20005','zip_20007','zip_20008','zip_20009','zip_20010','zip_20011',
               'zip_20012','zip_20015','zip_20017','zip_20018','zip_20019','zip_20020',
               'zip_20032','zip_20036','zip_20037','ldmonth_1','ldmonth_2','ldmonth_3',
               'ldmonth_4','ldmonth_5','ldmonth_6','ldmonth_7','ldmonth_8','ldmonth_9',
               'ldmonth_10','ldmonth_11','ldmonth_12','ldday_1','ldday_2','ldday_3',
               'ldday_4','ldday_5','ldday_6','ldday_7','ldday_8','ldday_9','ldday_10',
               'ldday_11','ldday_12','ldday_13','ldday_14','ldday_15','ldday_16','ldday_17',
               'ldday_18','ldday_19','ldday_20','ldday_21','ldday_22','ldday_23','ldday_24',
               'ldday_25','ldday_26','ldday_27','ldday_28','ldday_29','ldday_30','ldday_31',
               'ESSR_0.0','ESSR_1.0','ESSR_2.0','ESSR_3.0','ESSR_4.0','ESSR_5.0','HSSR_0.0',
               'HSSR_1.0','HSSR_2.0','HSSR_3.0','HSSR_4.0','MSSR_0.0','MSSR_1.0','MSSR_2.0',
               'MSSR_3.0','MSSR_4.0','MSSR_5.0']

In [11]:
# GridSearchCV parameters for pipeline
# MAE not used for rfr,etr,or gbr. Program would crash or hit memory limitations 
# if used with MAE.
# Parameters for estimator GridSearch

# MAE not used for rfr,etr,or gbr. Program would crash or hit memory limitations 
# if used with MSE.
rfr_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__n_estimators':[100],'estimator__criterion':['mse'],
              'estimator__max_features':['auto'],'estimator__min_samples_leaf':[1,2,5],
              'estimator__random_state':[1]}

etr_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__n_estimators':[100],'estimator__criterion':['mse'],
              'estimator__max_features':['auto'],'estimator__min_samples_leaf':[1,2,5],
              'estimator__random_state':[1]}

gbr_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__loss':['ls','lad','huber'],'estimator__learning_rate':[0.1],
              'estimator__n_estimators':[100],'estimator__criterion':['friedman_mse','mse'],
              'estimator__min_samples_leaf':[1,2,5],'estimator__max_features':['auto'],
              'estimator__random_state':[1]}

sgdr_params = {'decomposition__n_components':np.arange(1,11,1),
               'decomposition__whiten':[True],'decomposition__random_state':[1],
               'estimator__loss':['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive'],
               'estimator__alpha':[0.0001,0.001,0.01,0.1,1.0],
               'estimator__fit_intercept':[True,False],'estimator__n_iter':[5,10],     
               'estimator__random_state':[1]}

tsr_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__fit_intercept':[True,False],'estimator__max_iter':[400],     
              'estimator__random_state':[1]}

par_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__loss':['epsilon_insensitive','squared_epsilon_insensitive'],
              'estimator__C':[0.001,0.01,0.1,1.0],
              'estimator__fit_intercept':[True,False],'estimator__n_iter':[5,10],     
              'estimator__random_state':[1]}

hr_params = {'decomposition__n_components':np.arange(1,11,1),
             'decomposition__whiten':[True],'decomposition__random_state':[1],
             'estimator__alpha':[0.0001,0.001,0.01,0.1,1.0],
             'estimator__fit_intercept':[True,False],'estimator__max_iter':[200]}

lcv_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__eps':[0.001,0.01,0.1],'estimator__fit_intercept':[True,False],
              'estimator__cv':[4],'estimator__max_iter':[25000],
              'estimator__random_state':[1]}

rcv_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__alphas':[np.array([0.1,1.0,10.0])],
              'estimator__fit_intercept':[True,False],'estimator__cv':[4]}

encv_params = {'decomposition__n_components':np.arange(1,11,1),
               'decomposition__whiten':[True],'decomposition__random_state':[1],
               'estimator__l1_ratio':[0.2,0.4,0.6,0.8],'estimator__eps':[0.001,0.01,0.1],
               'estimator__fit_intercept':[True,False],'estimator__cv':[4],
               'estimator__max_iter':[25000],'estimator__random_state':[1]}

knnr_params = {'decomposition__n_components':np.arange(1,11,1),
               'decomposition__whiten':[True],'decomposition__random_state':[1],
               'estimator__n_neighbors':np.arange(2,22,2),
               'estimator__weights':['uniform','distance'],
               'estimator__algorithm':['ball_tree','kd_tree']}

svr_params = {'decomposition__n_components':np.arange(1,11,1),
              'decomposition__whiten':[True],'decomposition__random_state':[1],
              'estimator__C':[0.01,0.1,1.0],'estimator__kernel':['poly','rbf'],
              'estimator__degree':[1,2,3]}

In [33]:
def decomposition_feature_model_selection(df,model_estimator,model_estimator_str,pipe_params,CATEGORICAL):

    """
    The pipeline test-train-splits the dataset, and performs modelling with PCA for decomposition
    followed by GridSearchCV. Individual models are saved as dills, and the regression metrics
    are saved to csv.
    """
    
    # Start clock for run time
    start  = time.time()
    
    # Copy dataframe
    df_2 = df.copy()
    
    # Drop target column and qcut column for test-train-split
    df_2 = df_2.drop('DOMP', axis=1)
    df_2 = df_2.drop('qcut_DOMP', axis=1)
    
    # Test-train split. Using 70/30% split.
    X_train, X_test, y_train, y_test = train_test_split(df_2, df['DOMP'], train_size=0.70,
                                                    random_state=1)    
    
    # Standardizing training and testing data. Standardized separately to avoid information
    # leaking from the training set to the testing set. Categorical data not scaled.
    for i in X_train.columns.difference(CATEGORICAL):
        X_train[i] = StandardScaler().fit_transform(X_train[i].values.reshape(-1,1))

    for i in X_test.columns.difference(CATEGORICAL):
        X_test[i] = StandardScaler().fit_transform(X_test[i].values.reshape(-1,1))
    
    # Pipeline using PCA
    pipe = Pipeline([('decomposition', PCA()),
                      ('estimator', model_estimator)
                     ])
        
    # GridSearchCV for pipeline
    grid = GridSearchCV(pipe, pipe_params, cv = 12, n_jobs = -1, verbose=1)
    grid.fit(X_train, y_train)
    
    # Print out best score and respective parameters
    print('Best score from GridSearchCV Pipeline for '+model_estimator_str+' is ',grid.best_score_)
    #print "Best model parameters from GridSearch are ",grid.best_estimator_.get_params()
    
    # Save model to disk
    dill.dump(grid, open('PCA_model_selection_'+model_estimator_str+'_no prior_feature_selection', 'wb'))
    dill.dump(grid.best_estimator_, open('PCA_best_model_selection_'+model_estimator_str+'_no_prior_feature_selection', 'wb'))

    # Predicted target values
    y_pred = grid.predict(X_test)
   
    # Store regression metrics
    exp_var_score = explained_variance_score(y_test, y_pred)
    #r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)    

    # Create dataframe to save regression metrics
    df_combination = pd.DataFrame(columns = {'estimator','r2_score','exp_var_score',
                                             'mae','mse','process_time'})
    estimator_list_lst = []
    exp_var_score_lst = []
    r2_score_lst = []
    mae_lst = []
    mse_lst = []
    process_time_lst = []
    
    # Save metrics to separate lists for inclusion in dataframe. Saving directly to dataframe
    # resulted in typeerrors being flagged.
    estimator_list_lst.append(model_estimator_str)
    exp_var_score_lst.append(exp_var_score)
    r2_score_lst.append(grid.best_score_) # best_score here is the r2_score, since GridSearchCV
                                          # uses the default score metrics from the estimator 
    mae_lst.append(mae)
    mse_lst.append(mse)    
    process_time_lst.append(time.time()-start)
    
    # Add lists as series to dataframe, and save file to disk.
    df_combination['estimator'] = estimator_list_lst
    df_combination['exp_var_score'] = exp_var_score_lst
    df_combination['r2_score'] = r2_score_lst
    df_combination['mae'] = mae_lst
    df_combination['mse'] = mse_lst
    df_combination['process_time'] = process_time_lst
    df_combination.to_csv('PCA_model_selection_'+model_estimator_str+'_no prior_feature_selection.csv')
        
    # Print run time
    print("\nBuild and Validation took {:0.3f} seconds\n".format(time.time()-start))   

In [ ]:
decomposition_feature_model_selection(df_2,LassoCV(),'LCV',lcv_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,RidgeCV(),'RCV',rcv_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,ElasticNetCV(),'ENCV',encv_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,SGDRegressor(),'SGDR',sgdr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,TheilSenRegressor(),'TSR',tsr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,PassiveAggressiveRegressor(),'PAR',par_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,HuberRegressor(),'HR',hr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,RandomForestRegressor(),'RFR',rfr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,ExtraTreesRegressor(),'ETR',etr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,GradientBoostingRegressor(),'GBR',gbr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,KNeighborsRegressor(),'KNNR',knnr_params,CATEGORICAL)
#decomposition_feature_model_selection(df_2,SVR(),'SVR',svr_params,CATEGORICAL)

In [27]:
def plot_PCA_explained_variance_vs_n_components(df,model_estimator_str):
    # Import dill file of model
    model = dill.load(open('PCA_model_selection_'+model_estimator_str+'_no prior_feature_selection', 'rb'))
     
    # Plot explained variance vs. number of components
    plt.figure()
    plt.plot(model.named_steps['decomposition'].explained_variance_,linewidth=3)
    plt.title('Explained variance vs number of components',fontsize=18,fontweight='bold')
    plt.xlabel("Number of components",fontsize=14,fontweight='bold')
    plt.xticks(fontsize=14,fontweight='bold')
    plt.ylabel("Explained variance",fontsize=14,fontweight='bold')
    plt.yticks(fontsize=14,fontweight='bold')
    plt.legend(fontsize=16)
    plt.savefig('PCA_model_selection_'+model_estimator_str+'_plot_explained_variance_vs_number_of_components')
    plt.show()   

In [ ]:
plot_PCA_explained_variance_vs_n_components(df_3,'LCV')